In [2]:
%load_ext autoreload
%autoreload 2

# 0. Intro
1. 利用InferNet生成GT-3需要的params，由此可以计算MDN的input
# 1. preparations
## 1.1 全局设置

In [3]:
import torch
import os
import torch.nn as nn
import numpy as np
import random
import pandas as pd
from tqdm import tqdm
# from MLP.models import MLP_1_1
from MLP.models import MLP_GT3_2
from importlib import reload
from MLP.utils import setup_seed
from MLP.utils import load_checkpoint
from MLP.utils import get_InferNet_save_path

from MLP.my_collate_fn import my_collate_fn_GT2, my_collate_fn_3
from torch.utils.data import DataLoader, SequentialSampler, SubsetRandomSampler
from MLP.mydataset_GT2 import myDataset

from MLP.Config import config_GT_gen
reload(config_GT_gen)  # 必须reload！！
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

bound_alpha = torch.tensor([-0.3,0.3],device=device)
bound_labda = torch.tensor([0.01,18],device=device)

opt = config_GT_gen.DefaultConfig()

## 1.2 相关path

In [4]:
MODEL_NAME = "InferNet_GT3"

In [5]:
setup_seed(opt.seed)

dataset = myDataset(opt.train_path, opt.target_path_metric, opt.target_path_loss, opt.data_key_path)


data_idx = np.arange(0,dataset.__len__()).tolist()
print("seed=",opt.seed)
print("arti=",opt.ARTIFICIAL)
print("GT_w_Params=",opt.GT_w_Params)

seed= 62
arti= False
GT_w_Params= 2


# 2. dataloader
1. 每次返回1个data file，并且按照字典序

In [6]:
data_loader = DataLoader(dataset=dataset, batch_size=1, shuffle=False, num_workers=0, drop_last=False, sampler=SequentialSampler(data_idx), collate_fn=my_collate_fn_GT2)

# 3. 读取model并且生成

- 读取model

In [53]:
mlp = MLP_GT3_2()
if opt.ARTIFICIAL and opt.seed in [4,31,204,35,407]:
    mlp.load_state_dict(torch.load(opt.model_params_MLP))
elif not opt.ARTIFICIAL and opt.seed in [3,31,204,223,407]:
        mlp.load_state_dict(torch.load(opt.model_params_MLP))
else:
    model_path = get_InferNet_save_path(opt.ARTIFICIAL, opt.seed, opt.net_root_path, opt.noise_pct, MODEL_NAME)
    mlp, _ = load_checkpoint(model_path, mlp)
mlp = mlp.to(device=device)

- 或者读取data shift model

In [12]:
mlp = MLP_GT3_2()

mlp.load_state_dict(torch.load(opt.net_root_path + "NN_params_infer_GT3_sft_bidinc.pth"))
mlp = mlp.to(device=device)

In [13]:
params_arr = np.array([0.]*(dataset.__len__()*1)).reshape(-1,1)
params = pd.DataFrame(params_arr,columns = ['alpha'])

In [14]:
model=mlp.eval()
for batch_id, data in enumerate(data_loader):

    input_data, _,_, _= data
    # Do the inference
    input_data = input_data.to(device)

    Alpha= mlp(input_data)

    Alpha = Alpha.detach().cpu().numpy()

    params.loc[batch_id,'alpha'] = Alpha

print("Done")
print(params.head())

Done
      alpha
0 -0.011431
1 -0.011431
2 -0.011431
3 -0.011431
4 -0.011431


- save

In [56]:
print(opt.params_gen_path)

D:/Desktop/PROJ/PAProj\data/SA_PT/params_GT3_seed=626.csv


- data shift就存在：

In [15]:
shift_path = os.path.join(opt.data_root, "data/SA_PT/params_GT"+str(opt.GT_w_Params)+"_sft_bidinc" + ".csv")

shift_path

'D:/Desktop/PROJ/PAProj\\data/SA_PT/params_GT2_sft_bidinc.csv'

In [16]:
params.to_csv(shift_path,encoding="utf-8", header=True,index=False)


In [58]:
# params.to_csv(opt.params_gen_path,encoding="utf-8", header=True,index=False)